### Image stack intensity normalization
- Read image information (bitdepth)
- Intensity correction:
    - Gaussian blur (sigma = 400)
    - divide the original with the blurred image stack
- OpenCV: image normalization function

In [41]:
import cv2
import numpy as np
import tifffile
import os
from scipy.ndimage import gaussian_filter

In [2]:
os.getcwd()

'/Users/wanqing.yu/Documents/GitHub/intensity normalization stitched'

In [6]:
# read the source image and get the bit depth; define output location
image_src = "/Users/wanqing.yu/Documents/GitHub/intensity normalization stitched/materialized_tiffs/stack_10_flat_affine.tif"
image_src_test = "/Users/wanqing.yu/Documents/GitHub/intensity normalization stitched/materialized_tiffs/test_stack0_flat-1.tif"
# # Load the 3D TIFF stack
# with tifffile.TiffFile(image_src_test) as tif:
#     # Access the TIFF file's BitDepth from metadata
#     image_bitDepth = tif.pages[0].tags['BitsPerSample'].value

In [7]:
# Load the 3D TIFF stack
stack = tifffile.imread(image_src_test)

# 'stack' is now a NumPy array where each element is a 2D image slice
# You can access individual slices like stack[0], stack[1], etc.

# Get information about the stack dimensions
num_slices, height, width = stack.shape
print(f"Number of slices: {num_slices}")
print(f"Image dimensions: {width}x{height}")

TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'


Number of slices: 32
Image dimensions: 2000x780


In [8]:
stack[31]

array([[   0,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       ...,
       [   0,    0,    0, ..., 1372, 1682, 2021],
       [   0,    0,    0, ..., 1519, 1845, 1948],
       [   0,    0,    0, ..., 1893, 2035, 2177]], dtype=uint16)

In [30]:
# 2D Gaussian Blur

def intensity_correction_2d(stack, sigma=500, kernel_size=(2005, 2005), epsilon=1e-6):
    num_slices = stack.shape[0]
    blurred_stack = np.empty_like(stack)
    blurred_stack_with_epsilon = np.empty_like(stack, dtype=float)

    for i in range(num_slices):
        blurred_slice = cv2.GaussianBlur(stack[i], kernel_size, sigma)
        blurred_stack[i] = blurred_slice
        blurred_stack_with_epsilon[i] = blurred_stack[i].astype(np.float32) + epsilon

    result_image = stack.astype(np.float16) / blurred_stack_with_epsilon.astype(np.float16)

    return result_image

In [27]:
result_stack_2d = intensity_correction_2d(stack)
tifffile.imsave('intensity_corrected_stack_2d.tif', result_stack_2d)

/var/folders/w4/p4yxd52x0mb02jrynpfgj3sw0000gr/T/ipykernel_87735/4064032709.py:15: RuntimeWarning: overflow encountered in cast
  result_image = stack.astype(np.float16) / blurred_stack_with_epsilon.astype(np.float16)


In [42]:
# 3D Gaussian Blur

def intensity_correction_3d(stack, sigma=400):
    blurred_stack = gaussian_filter(stack, sigma=sigma)
    epsilon = 1e-6
    result_image = stack.astype(np.float16) / (blurred_stack.astype(np.float16) + epsilon)
    return result_image

# Example usage for 3D intensity correction:
# Assuming your_input_stack is a 3D NumPy array
# result_stack_3d = intensity_correction_3d(your_input_stack)
# tifffile.imsave('intensity_corrected_stack_3d.tif', result_stack_3d)


In [ ]:
#sigma = (1000,1000,20)
result_stack_3d = intensity_correction_3d(stack)
tifffile.imsave('intensity_corrected_stack_3d.tif', result_stack_3d)